## Data Cleaning and Transformation


I have a dataset with 10,000 startup-like entities. My goal is to filter and transform this dataset according to our criteria with the use of Python. Every transformation must be conducted on the output of the previous one so they compound on each other, but generally the filtering order is not strict.

There are 5 files:
-test_task.csv is the core dataset that you will be transforming.
-db_domains.csv contains the list of the domains and uuids of entities already in our database.
-banned_domains.csv contains the list of top-level domains we don't want in our database.
-banned_words.csv contains the list of words that we don't associate with startups.
-funding_data.csv contains the dataset with investments for various entities with stages.

Below are the steps I have to do, using these files:
1) Remove all rows with duplicate uuids or websites (leave only one from every duplicate group).
2) Remove the rows with no website, description, category or location
3) Remove all rows where valuation is equal to or exceeds 1 billion dollars.
4) Remove the rows with entities which contain uuids or websites from db_domains.csv.
5) Remove the rows where the top-level domains of the website are in the banned_domains.csv.
6) Remove the rows with entities the descriptions of which contain any words from banned_words.csv.
7) The column "categories" contains lists of dictionaries with names of categories for every entity. Create a new column "categories_list" with lists of extracted names of categories from these lists of dictionaries (key 'value').
8) The column "location" contains dictionaries with lists of locations at different scale, namely city, region and country, plus continent. Extract cities, regions and countries for every entity and put every list in a separate column named accordingly.
9) Sum up the investments from funding_data.csv per entity (we recommend using uuid) and add a column in the main dataframe with total investment for every entity based on what you calculated. Not all entities will have investments and not all investment data will be used, so don't worry about blank spaces.

10) Save the resulting dataframe after all the steps into a csv, as well as your Python code (as a .py or .ipynb file).


## Importing the necessary libraries and reading all the dataset

In [72]:
# Importing pandas to read the data  in a DataFrame
import pandas as pd

# Importing warnings To ignore warnings from the code
import warnings
warnings.filterwarnings("ignore")


In [73]:
# Reading all the dataset required to be cleaned and filtered 
test_task = pd.read_csv(r"C:\Users\Administrator\Downloads\Test\test_task.csv")
db_domains = pd.read_csv(r"C:\Users\Administrator\Downloads\Test\db_domains.csv")
banned_domains = pd.read_csv(r"C:\Users\Administrator\Downloads\Test\banned_domains.csv")
banned_words = pd.read_csv(r"C:\Users\Administrator\Downloads\Test\banned_words.csv")
funding_data = pd.read_csv(r"C:\Users\Administrator\Downloads\Test\funding_data.csv")


In [74]:
# To view the first 5 rows of the test_task dataset
test_task.head()

,Unnamed: 0,uuid,website,Name,description,valuation_usd,categories,location
0,0,5639c94c-408b-450a-9694-df238f6d9da9,https://www.perplexity.ai,Perplexity AI,Perplexity is a search engine platform that us...,5.200000e+08,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'san-francisco-california', 'uu..."
1,1,00e32424-3cc3-4798-8361-78914c212fb0,https://mistral.ai,Mistral AI,Mistral AI is a platform that assembles team t...,1.999462e+09,"[{'entity_def_id': 'category', 'permalink': 'a...",NaN
2,2,c8326e69-b364-44af-a7a7-1f9239d532f1,https://robinai.co.uk,Robin AI,Robin AI is a legal infrastructure business th...,NaN,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'london-england', 'uuid': 'aad1..."
3,3,cf2c678c-b81a-80c3-10d1-9c5e76448e51,https://www.openai.com,OpenAI,OpenAI is an AI research and deployment compan...,2.900000e+10,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'san-francisco-california', 'uu..."
4,4,0f3fd8ce-9fe3-d6bd-d243-ffdf2fcfb012,http://www.startengine.com,StartEngine,StartEngine is an equity crowdfunding platform...,1.200000e+08,"[{'entity_def_id': 'category', 'permalink': 'c...","[{'permalink': 'los-angeles-california', 'uuid..."


In [75]:
# To view shape of the dataset
test_task.shape

(10000, 8)

### 1. Remove all rows with duplicate uuids or websites (leave only one from every duplicate group)

In [76]:
# checking for duplicates
test_task.duplicated(subset=['uuid', 'website']).sum()


0

No duplicated rows found in uuids and website column

### 2. Remove the rows with no website, description, category or location


In [77]:
# counting empty rows in the each column in test_task 
test_task.isna().sum()

Unnamed: 0          0
uuid                0
website            23
Name                0
description       168
valuation_usd    8080
categories         15
location            5
dtype: int64

In [78]:
# checking for rows with no website, description, category or location
test_task[['website', 'description', 'categories', 'location']].isna().sum()

website         23
description    168
categories      15
location         5
dtype: int64

In [79]:
# dropping these empty rows
test_task = test_task.dropna(subset=['website', 'description', 'categories', 'location'])

# Rechecking for rows with no website, description, category or location
test_task[['website', 'description', 'categories', 'location']].isna().sum()

website        0
description    0
categories     0
location       0
dtype: int64

### 3. Remove all rows where valuation is equal to or exceeds 1 billion dollars.

In [80]:
# Filtering out rows with valuation >= 1 billion
test_task = test_task[test_task['valuation_usd'] < 1_000_000_000]

# view the new dataset
test_task.head()

,Unnamed: 0,uuid,website,Name,description,valuation_usd,categories,location
0,0,5639c94c-408b-450a-9694-df238f6d9da9,https://www.perplexity.ai,Perplexity AI,Perplexity is a search engine platform that us...,520000000.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'san-francisco-california', 'uu..."
4,4,0f3fd8ce-9fe3-d6bd-d243-ffdf2fcfb012,http://www.startengine.com,StartEngine,StartEngine is an equity crowdfunding platform...,120000000.0,"[{'entity_def_id': 'category', 'permalink': 'c...","[{'permalink': 'los-angeles-california', 'uuid..."
5,5,99cc593e-96ee-48fe-824a-0ebc12a94a63,https://bitscrunch.com,bitsCrunch,AI-enhanced decentralized data networks delive...,58850000.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'munich-bayern', 'uuid': '6469a..."
8,8,dc6b7b23-1e15-49ed-accc-5d168c32c368,https://lumalabs.ai,Luma AI,Luma AI is a generative AI startup that enable...,250000000.0,"[{'entity_def_id': 'category', 'permalink': '3...","[{'permalink': 'palo-alto-california', 'uuid':..."
12,12,350922d5-ca52-cfac-1091-49be0ac076f1,https://www.1x.tech,1X,1X designs and develops humanoid robots to ope...,47621207.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'moss-ostfold', 'uuid': '956600..."


In [81]:
# view the new shape of the dataset
test_task.shape

(1311, 8)

#### 4. Remove the rows with entities which contain uuids or websites from db_domains.csv.

In [82]:
# view the db_domains dataset
db_domains

,id,url,uuid_cb
0,3,https://www.cloudsafe.com,5fd90772-a6f6-16a7-e263-8a00c9cb186e
1,7,http://www.baydin.com,8b54de34-f13a-6bcd-cd32-c2bb36240b2e
2,15,http://www.weedconnection.com,3e45c02a-425c-614e-1de7-6ef51c749cd3
3,19,http://www.graphicly.com,94e2370c-cdd0-586d-e620-480c64c082ed
4,24,http://appstores.com,cf3998e9-79a2-ec85-b6f6-6ebb9afb5d8d
...,...,...,...
96134,428428,www.waverity.ai,NaN
96135,428429,www.bahshish.me,NaN
96136,428430,https://www.e-sayar.az/,NaN
96137,428431,https://picnichr.com,NaN


In [83]:
# Extracting unique values from 'uuid_cb' and 'url' columns, if they exist
uuid_set = set(db_domains.get('uuid_cb', []))
website_set = set(db_domains.get('url', []))

# Combine these sets into one
db_domains_set = uuid_set.union(website_set)

# To check if 'uuid' and 'website' columns exist in test_task
if {'uuid', 'website'}.issubset(test_task.columns):
    # Filter out rows from `test_task` where 'uuid' or 'website' is in db_domains_set
    test_task = test_task[~test_task['uuid'].isin(db_domains_set) & ~test_task['website'].isin(db_domains_set)]

# View the new dataset
test_task

,Unnamed: 0,uuid,website,Name,description,valuation_usd,categories,location
0,0,5639c94c-408b-450a-9694-df238f6d9da9,https://www.perplexity.ai,Perplexity AI,Perplexity is a search engine platform that us...,520000000.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'san-francisco-california', 'uu..."
5,5,99cc593e-96ee-48fe-824a-0ebc12a94a63,https://bitscrunch.com,bitsCrunch,AI-enhanced decentralized data networks delive...,58850000.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'munich-bayern', 'uuid': '6469a..."
8,8,dc6b7b23-1e15-49ed-accc-5d168c32c368,https://lumalabs.ai,Luma AI,Luma AI is a generative AI startup that enable...,250000000.0,"[{'entity_def_id': 'category', 'permalink': '3...","[{'permalink': 'palo-alto-california', 'uuid':..."
19,19,49a11bdf-3dad-4991-b241-0662b11dd40b,https://www.nabla.com,Nabla,"Nabla is the developer of Nabla Copilot, an am...",180000000.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'paris-ile-de-france', 'uuid': ..."
30,30,de7c1608-59de-46f6-8902-3aa91d881470,https://aleph-alpha.com,Aleph Alpha,Aleph Alpha transforms human-machine interacti...,491239560.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'heidelberg-baden-wurttemberg',..."
...,...,...,...,...,...,...,...,...
9852,852,933c3357-ef87-43f3-9179-857b66f758c1,https://markopolo.ai/,Markopolo,Markopolo is a complete paid marketing suite t...,3000000.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'wilmington-delaware', 'uuid': ..."
9872,872,092d0735-eb4e-4847-ae30-79287e222ded,https://www.medulance.com/,Medulance,Medulance is a GPS-based technology platform t...,12075084.0,"[{'entity_def_id': 'category', 'permalink': 'g...","[{'permalink': 'new-delhi-delhi', 'uuid': '7b2..."
9946,946,4cebcabf-bfbd-43cb-b58e-e6aa45171ee2,https://limacharlie.io,LimaCharlie,Security Infrastructure as a Service (SIaaS).\...,24000000.0,"[{'entity_def_id': 'category', 'permalink': 'c...","[{'permalink': 'walnut-california', 'uuid': 'a..."
9951,951,248eb9ad-61b2-4c55-ae5a-e544dc055bdb,https://updf.com,"Superace Software Technologies Co., Ltd.",Superace Software develops cross-platform docu...,19500836.0,"[{'entity_def_id': 'category', 'permalink': 's...","[{'permalink': 'shenzhen-guangdong', 'uuid': '..."


### 5. Remove the rows where the top-level domains of the website are in the banned_domains.csv.

In [84]:
# view the banned domains datset
banned_domains

,urls
0,.google
1,.yandex
2,.amazon
3,.adobe
4,.att
5,.medium
6,.baidu
7,.salesforce
8,.facebook
9,.microsoft


In [85]:
# Extract banned top-level domains(tld) (remove leading dots if present)
banned_tlds = banned_domains['urls'].str.lstrip('.').tolist()

# Function to check if a website contains any banned TLD
def contains_banned_tld(url, banned_tlds):
    # Split the URL by '.' and check if the last part is in banned TLDs
    return any(tld in url.lower().split('.')[-1] for tld in banned_tlds)

# Apply the function to filter out rows by creating a temporary column for top-level banned do
test_task['banned_tld'] = test_task['website'].apply(lambda x: contains_banned_tld(x, banned_tlds))
test_task['banned_tld'].sum()

4

Four banned domains to be removed

In [86]:
# Keep only rows where 'tld' is False
test_task = test_task[~test_task['banned_tld']]


# Drop the temporary 'tld' column
test_task = test_task.drop(columns=['banned_tld'])

# view new dataset
test_task

,Unnamed: 0,uuid,website,Name,description,valuation_usd,categories,location
0,0,5639c94c-408b-450a-9694-df238f6d9da9,https://www.perplexity.ai,Perplexity AI,Perplexity is a search engine platform that us...,520000000.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'san-francisco-california', 'uu..."
5,5,99cc593e-96ee-48fe-824a-0ebc12a94a63,https://bitscrunch.com,bitsCrunch,AI-enhanced decentralized data networks delive...,58850000.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'munich-bayern', 'uuid': '6469a..."
8,8,dc6b7b23-1e15-49ed-accc-5d168c32c368,https://lumalabs.ai,Luma AI,Luma AI is a generative AI startup that enable...,250000000.0,"[{'entity_def_id': 'category', 'permalink': '3...","[{'permalink': 'palo-alto-california', 'uuid':..."
19,19,49a11bdf-3dad-4991-b241-0662b11dd40b,https://www.nabla.com,Nabla,"Nabla is the developer of Nabla Copilot, an am...",180000000.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'paris-ile-de-france', 'uuid': ..."
30,30,de7c1608-59de-46f6-8902-3aa91d881470,https://aleph-alpha.com,Aleph Alpha,Aleph Alpha transforms human-machine interacti...,491239560.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'heidelberg-baden-wurttemberg',..."
...,...,...,...,...,...,...,...,...
9852,852,933c3357-ef87-43f3-9179-857b66f758c1,https://markopolo.ai/,Markopolo,Markopolo is a complete paid marketing suite t...,3000000.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'wilmington-delaware', 'uuid': ..."
9872,872,092d0735-eb4e-4847-ae30-79287e222ded,https://www.medulance.com/,Medulance,Medulance is a GPS-based technology platform t...,12075084.0,"[{'entity_def_id': 'category', 'permalink': 'g...","[{'permalink': 'new-delhi-delhi', 'uuid': '7b2..."
9946,946,4cebcabf-bfbd-43cb-b58e-e6aa45171ee2,https://limacharlie.io,LimaCharlie,Security Infrastructure as a Service (SIaaS).\...,24000000.0,"[{'entity_def_id': 'category', 'permalink': 'c...","[{'permalink': 'walnut-california', 'uuid': 'a..."
9951,951,248eb9ad-61b2-4c55-ae5a-e544dc055bdb,https://updf.com,"Superace Software Technologies Co., Ltd.",Superace Software develops cross-platform docu...,19500836.0,"[{'entity_def_id': 'category', 'permalink': 's...","[{'permalink': 'shenzhen-guangdong', 'uuid': '..."


### 6. Remove the rows with entities the descriptions of which contain any words from banned_words.csv.

In [87]:
# View the banned words dataset
banned_words

,words
0,Accelerator
1,Association
2,Brewing
3,Business School
4,Capital
5,Clinic
6,College
7,Entrepreneurship
8,Hospital
9,Hospitality


In [88]:
# Extract banned words into a set for efficient lookup
banned_words_set = set(banned_words['words'])

# Function to check if a description contains any banned words
def contains_banned_word(description):
    return any(word in description for word in banned_words_set)

# Apply the function to check each description for banned words, creating a boolean Series
banned_word_mask = test_task['description'].apply(contains_banned_word)

# Filter the DataFrame to include only rows with banned words in the description
rows_with_banned_words = test_task[banned_word_mask]

# Count the number of non-null entries in each column of the filtered DataFrame
rows_with_banned_words['description'].count()


37

Thirty Seven rows with banned words found

In [89]:
# Removing rows with banned words from the DataFrame
test_task = test_task[~banned_word_mask]
test_task

,Unnamed: 0,uuid,website,Name,description,valuation_usd,categories,location
0,0,5639c94c-408b-450a-9694-df238f6d9da9,https://www.perplexity.ai,Perplexity AI,Perplexity is a search engine platform that us...,520000000.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'san-francisco-california', 'uu..."
5,5,99cc593e-96ee-48fe-824a-0ebc12a94a63,https://bitscrunch.com,bitsCrunch,AI-enhanced decentralized data networks delive...,58850000.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'munich-bayern', 'uuid': '6469a..."
8,8,dc6b7b23-1e15-49ed-accc-5d168c32c368,https://lumalabs.ai,Luma AI,Luma AI is a generative AI startup that enable...,250000000.0,"[{'entity_def_id': 'category', 'permalink': '3...","[{'permalink': 'palo-alto-california', 'uuid':..."
30,30,de7c1608-59de-46f6-8902-3aa91d881470,https://aleph-alpha.com,Aleph Alpha,Aleph Alpha transforms human-machine interacti...,491239560.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'heidelberg-baden-wurttemberg',..."
82,82,71be95da-83de-42c7-9eeb-b3003dc392b8,https://www.jupiter.money,Jupiter,Jupiter is a digital banking app with one simp...,711000000.0,"[{'entity_def_id': 'category', 'permalink': 'b...","[{'permalink': 'mumbai-maharashtra', 'uuid': '..."
...,...,...,...,...,...,...,...,...
9852,852,933c3357-ef87-43f3-9179-857b66f758c1,https://markopolo.ai/,Markopolo,Markopolo is a complete paid marketing suite t...,3000000.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'wilmington-delaware', 'uuid': ..."
9872,872,092d0735-eb4e-4847-ae30-79287e222ded,https://www.medulance.com/,Medulance,Medulance is a GPS-based technology platform t...,12075084.0,"[{'entity_def_id': 'category', 'permalink': 'g...","[{'permalink': 'new-delhi-delhi', 'uuid': '7b2..."
9946,946,4cebcabf-bfbd-43cb-b58e-e6aa45171ee2,https://limacharlie.io,LimaCharlie,Security Infrastructure as a Service (SIaaS).\...,24000000.0,"[{'entity_def_id': 'category', 'permalink': 'c...","[{'permalink': 'walnut-california', 'uuid': 'a..."
9951,951,248eb9ad-61b2-4c55-ae5a-e544dc055bdb,https://updf.com,"Superace Software Technologies Co., Ltd.",Superace Software develops cross-platform docu...,19500836.0,"[{'entity_def_id': 'category', 'permalink': 's...","[{'permalink': 'shenzhen-guangdong', 'uuid': '..."


### 7. The column "categories" contains lists of dictionaries with names of categories for every entity. Create a new column "categories_list" with lists of extracted names of categories from these lists of dictionaries (key 'value').

In [90]:
# To view first row of the 'categories' column
test_task.iloc[0]['categories']

"[{'entity_def_id': 'category', 'permalink': 'artificial-intelligence', 'uuid': 'c4d8caf3-5fe7-359b-f9f2-2d708378e4ee', 'value': 'Artificial Intelligence (AI)'}, {'entity_def_id': 'category', 'permalink': 'chatbot-f0f7', 'uuid': 'e78fbf98-2d83-4b41-b8a9-099a912be5a3', 'value': 'Chatbot'}, {'entity_def_id': 'category', 'permalink': 'generative-ai', 'uuid': '8314a534-8f18-4ee0-869b-05bc6e46258e', 'value': 'Generative AI'}, {'entity_def_id': 'category', 'permalink': 'search-engine-0d39', 'uuid': '0d39287e-5b13-77a6-9702-74294daded43', 'value': 'Search Engine'}, {'entity_def_id': 'category', 'permalink': 'software', 'uuid': 'c08b5441-a05b-9777-b7a6-012728caddd9', 'value': 'Software'}]"

In [91]:
# Import ast module for safely evaluating strings containing Python literals
import ast  

# Function to convert string representation of list of dictionaries to lists of 'value' fields
def extract_category_names(category_string):
    try:
        # Converting string representation of list of dictionaries to actual list of dictionaries
        categories = ast.literal_eval(category_string)
        
        # Extract 'value' from each dictionary in the list
        return [category['value'] for category in categories]
    except (ValueError, SyntaxError) as e:
        print(f"Error parsing categories: {e}")
        return []
    except Exception as e:
        print(f"Unexpected error: {e}")
        return []

# Applying the function to create the new column
test_task['categories_list'] = test_task['categories'].apply(extract_category_names)

# View the category columns
test_task[['categories', 'categories_list']].head()


,categories,categories_list
0,"[{'entity_def_id': 'category', 'permalink': 'a...","[Artificial Intelligence (AI), Chatbot, Genera..."
5,"[{'entity_def_id': 'category', 'permalink': 'a...","[Analytics, Artificial Intelligence (AI), Bloc..."
8,"[{'entity_def_id': 'category', 'permalink': '3...","[3D Technology, Artificial Intelligence (AI), ..."
30,"[{'entity_def_id': 'category', 'permalink': 'a...","[Artificial Intelligence (AI), Generative AI, ..."
82,"[{'entity_def_id': 'category', 'permalink': 'b...","[Banking, Consulting, Finance, Financial Servi..."



### 8. The column "location" contains dictionaries with lists of locations at different scale, namely city, region and country, plus continent. Extract cities, regions and countries for every entity and put every list in a separate column named accordingly.

In [92]:
# View the filtered FataFrame
test_task

,Unnamed: 0,uuid,website,Name,description,valuation_usd,categories,location,categories_list
0,0,5639c94c-408b-450a-9694-df238f6d9da9,https://www.perplexity.ai,Perplexity AI,Perplexity is a search engine platform that us...,520000000.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'san-francisco-california', 'uu...","[Artificial Intelligence (AI), Chatbot, Genera..."
5,5,99cc593e-96ee-48fe-824a-0ebc12a94a63,https://bitscrunch.com,bitsCrunch,AI-enhanced decentralized data networks delive...,58850000.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'munich-bayern', 'uuid': '6469a...","[Analytics, Artificial Intelligence (AI), Bloc..."
8,8,dc6b7b23-1e15-49ed-accc-5d168c32c368,https://lumalabs.ai,Luma AI,Luma AI is a generative AI startup that enable...,250000000.0,"[{'entity_def_id': 'category', 'permalink': '3...","[{'permalink': 'palo-alto-california', 'uuid':...","[3D Technology, Artificial Intelligence (AI), ..."
30,30,de7c1608-59de-46f6-8902-3aa91d881470,https://aleph-alpha.com,Aleph Alpha,Aleph Alpha transforms human-machine interacti...,491239560.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'heidelberg-baden-wurttemberg',...","[Artificial Intelligence (AI), Generative AI, ..."
82,82,71be95da-83de-42c7-9eeb-b3003dc392b8,https://www.jupiter.money,Jupiter,Jupiter is a digital banking app with one simp...,711000000.0,"[{'entity_def_id': 'category', 'permalink': 'b...","[{'permalink': 'mumbai-maharashtra', 'uuid': '...","[Banking, Consulting, Finance, Financial Servi..."
...,...,...,...,...,...,...,...,...,...
9852,852,933c3357-ef87-43f3-9179-857b66f758c1,https://markopolo.ai/,Markopolo,Markopolo is a complete paid marketing suite t...,3000000.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'wilmington-delaware', 'uuid': ...","[Advertising, Advertising Platforms, Generativ..."
9872,872,092d0735-eb4e-4847-ae30-79287e222ded,https://www.medulance.com/,Medulance,Medulance is a GPS-based technology platform t...,12075084.0,"[{'entity_def_id': 'category', 'permalink': 'g...","[{'permalink': 'new-delhi-delhi', 'uuid': '7b2...","[GPS, Medical, Mobile Apps]"
9946,946,4cebcabf-bfbd-43cb-b58e-e6aa45171ee2,https://limacharlie.io,LimaCharlie,Security Infrastructure as a Service (SIaaS).\...,24000000.0,"[{'entity_def_id': 'category', 'permalink': 'c...","[{'permalink': 'walnut-california', 'uuid': 'a...","[Cloud Security, Cyber Security, Information T..."
9951,951,248eb9ad-61b2-4c55-ae5a-e544dc055bdb,https://updf.com,"Superace Software Technologies Co., Ltd.",Superace Software develops cross-platform docu...,19500836.0,"[{'entity_def_id': 'category', 'permalink': 's...","[{'permalink': 'shenzhen-guangdong', 'uuid': '...",[Software]


In [93]:
# view the location column
test_task['location']

0       [{'permalink': 'san-francisco-california', 'uu...
5       [{'permalink': 'munich-bayern', 'uuid': '6469a...
8       [{'permalink': 'palo-alto-california', 'uuid':...
30      [{'permalink': 'heidelberg-baden-wurttemberg',...
82      [{'permalink': 'mumbai-maharashtra', 'uuid': '...
                              ...                        
9852    [{'permalink': 'wilmington-delaware', 'uuid': ...
9872    [{'permalink': 'new-delhi-delhi', 'uuid': '7b2...
9946    [{'permalink': 'walnut-california', 'uuid': 'a...
9951    [{'permalink': 'shenzhen-guangdong', 'uuid': '...
9989    [{'permalink': 'gurgaon-haryana', 'uuid': '43a...
Name: location, Length: 431, dtype: object

In [94]:
# Inspect the contents of the first row
test_task.iloc[0]['location']

"[{'permalink': 'san-francisco-california', 'uuid': '528f5e3c-90d1-1111-5d1c-2e4ff979d58e', 'location_type': 'city', 'entity_def_id': 'location', 'value': 'San Francisco'}, {'permalink': 'california-united-states', 'uuid': 'eb879a83-c91a-121e-0bb8-829782dbcf04', 'location_type': 'region', 'entity_def_id': 'location', 'value': 'California'}, {'permalink': 'united-states', 'uuid': 'f110fca2-1055-99f6-996d-011c198b3928', 'location_type': 'country', 'entity_def_id': 'location', 'value': 'United States'}, {'permalink': 'north-america', 'uuid': 'b25caef9-a1b8-3a5d-6232-93b2dfb6a1d1', 'location_type': 'continent', 'entity_def_id': 'location', 'value': 'North America'}]"

In [95]:
# Inspect the contents of the 2nd row
test_task.iloc[1]['location']

"[{'permalink': 'munich-bayern', 'uuid': '6469af1b-6dbf-1e9e-17d9-e4f39a86aa95', 'location_type': 'city', 'entity_def_id': 'location', 'value': 'Munich'}, {'permalink': 'bayern-germany', 'uuid': 'eec7d129-caca-eeae-1bd7-d8449e855904', 'location_type': 'region', 'entity_def_id': 'location', 'value': 'Bayern'}, {'permalink': 'germany', 'uuid': '6085b4bf-b18a-1763-a04e-fdde3f6aba94', 'location_type': 'country', 'entity_def_id': 'location', 'value': 'Germany'}, {'permalink': 'europe', 'uuid': '6106f5dc-823e-5da8-40d7-51612c0b2c4e', 'location_type': 'continent', 'entity_def_id': 'location', 'value': 'Europe'}]"

In [96]:
# Function to extract cities, regions and countries from a list of loaction dictionaries
def extract_locations(location_data):
    # Convert the string representation of list of dictionaries to a list of dictionaries
    if isinstance(location_data, str):
        try:
            location_data = ast.literal_eval(location_data)
        except (ValueError, SyntaxError):
            return '', '', ''  # Return empty strings if conversion fails

    # Extract locations based on location_type
    cities = ', '.join(loc['value'] for loc in location_data if loc['location_type'] == 'city')
    regions = ', '.join(loc['value'] for loc in location_data if loc['location_type'] == 'region')
    countries = ', '.join(loc['value'] for loc in location_data if loc['location_type'] == 'country')
    
    return cities, regions, countries

# Apply the function to create new columns for cities, regions, and countries
test_task[['cities', 'regions', 'countries']] = test_task['location'].apply(lambda x: pd.Series(extract_locations(x)))

# Inspecting the result
test_task[['location', 'cities', 'regions', 'countries']].head()


,location,cities,regions,countries
0,"[{'permalink': 'san-francisco-california', 'uu...",San Francisco,California,United States
5,"[{'permalink': 'munich-bayern', 'uuid': '6469a...",Munich,Bayern,Germany
8,"[{'permalink': 'palo-alto-california', 'uuid':...",Palo Alto,California,United States
30,"[{'permalink': 'heidelberg-baden-wurttemberg',...",Heidelberg,Baden-Wurttemberg,Germany
82,"[{'permalink': 'mumbai-maharashtra', 'uuid': '...",Mumbai,Maharashtra,India


In [97]:
# View the updated dataframe
test_task

,Unnamed: 0,uuid,website,Name,description,valuation_usd,categories,location,categories_list,cities,regions,countries
0,0,5639c94c-408b-450a-9694-df238f6d9da9,https://www.perplexity.ai,Perplexity AI,Perplexity is a search engine platform that us...,520000000.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'san-francisco-california', 'uu...","[Artificial Intelligence (AI), Chatbot, Genera...",San Francisco,California,United States
5,5,99cc593e-96ee-48fe-824a-0ebc12a94a63,https://bitscrunch.com,bitsCrunch,AI-enhanced decentralized data networks delive...,58850000.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'munich-bayern', 'uuid': '6469a...","[Analytics, Artificial Intelligence (AI), Bloc...",Munich,Bayern,Germany
8,8,dc6b7b23-1e15-49ed-accc-5d168c32c368,https://lumalabs.ai,Luma AI,Luma AI is a generative AI startup that enable...,250000000.0,"[{'entity_def_id': 'category', 'permalink': '3...","[{'permalink': 'palo-alto-california', 'uuid':...","[3D Technology, Artificial Intelligence (AI), ...",Palo Alto,California,United States
30,30,de7c1608-59de-46f6-8902-3aa91d881470,https://aleph-alpha.com,Aleph Alpha,Aleph Alpha transforms human-machine interacti...,491239560.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'heidelberg-baden-wurttemberg',...","[Artificial Intelligence (AI), Generative AI, ...",Heidelberg,Baden-Wurttemberg,Germany
82,82,71be95da-83de-42c7-9eeb-b3003dc392b8,https://www.jupiter.money,Jupiter,Jupiter is a digital banking app with one simp...,711000000.0,"[{'entity_def_id': 'category', 'permalink': 'b...","[{'permalink': 'mumbai-maharashtra', 'uuid': '...","[Banking, Consulting, Finance, Financial Servi...",Mumbai,Maharashtra,India
...,...,...,...,...,...,...,...,...,...,...,...,...
9852,852,933c3357-ef87-43f3-9179-857b66f758c1,https://markopolo.ai/,Markopolo,Markopolo is a complete paid marketing suite t...,3000000.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'wilmington-delaware', 'uuid': ...","[Advertising, Advertising Platforms, Generativ...",Wilmington,Delaware,United States
9872,872,092d0735-eb4e-4847-ae30-79287e222ded,https://www.medulance.com/,Medulance,Medulance is a GPS-based technology platform t...,12075084.0,"[{'entity_def_id': 'category', 'permalink': 'g...","[{'permalink': 'new-delhi-delhi', 'uuid': '7b2...","[GPS, Medical, Mobile Apps]",New Delhi,Delhi,India
9946,946,4cebcabf-bfbd-43cb-b58e-e6aa45171ee2,https://limacharlie.io,LimaCharlie,Security Infrastructure as a Service (SIaaS).\...,24000000.0,"[{'entity_def_id': 'category', 'permalink': 'c...","[{'permalink': 'walnut-california', 'uuid': 'a...","[Cloud Security, Cyber Security, Information T...",Walnut,California,United States
9951,951,248eb9ad-61b2-4c55-ae5a-e544dc055bdb,https://updf.com,"Superace Software Technologies Co., Ltd.",Superace Software develops cross-platform docu...,19500836.0,"[{'entity_def_id': 'category', 'permalink': 's...","[{'permalink': 'shenzhen-guangdong', 'uuid': '...",[Software],Shenzhen,Guangdong,China


### 9. Sum up the investments from funding_data.csv per entity (we recommend using uuid) and add a column in the main dataframe with total investment for every entity based on what you calculated. Not all entities will have investments and not all investment data will be used, so don't worry about blank spaces.

In [98]:
# view the funding dataset
funding_data

,uuid,funding_round,funding_date,money_raised,company_name
0,b7947f18-b199-45ac-b7da-66f5c52fcfbc,Series D - Hugging Face,23/08/2023,235000000.0,Hugging Face
1,46ace8da-8b4f-e267-757c-158bae1c8c66,Series D - Neuralink,07/08/2023,280000000.0,Neuralink
2,e10aaff2-4d89-46d4-820b-b4f64b8d42ca,Corporate Round - Anthropic,13/08/2023,100000000.0,Anthropic
3,2b67d957-4f54-4d8d-8202-2bfa4ba681bd,Series A - LemFi,24/08/2023,33000000.0,LemFi
4,df8fc290-c5b9-a3c4-b68c-f9cb6b2971bb,Series F - Gympass,23/08/2023,85000000.0,Gympass
...,...,...,...,...,...
152644,NaN,NaN,NaN,NaN,NaN
152645,NaN,NaN,NaN,NaN,NaN
152646,NaN,NaN,NaN,NaN,NaN
152647,NaN,NaN,NaN,NaN,NaN


In [99]:
# Counting missing entries in money raised column
funding_data['money_raised'].isna().sum()

118905

In [100]:
# Replace missing values in 'money_raised' column with 0
funding_data['money_raised'] = funding_data['money_raised'].fillna(0)

# Grouping the data by 'uuid' and sum the 'money_raised' for each group, then reset the index
total_investments = funding_data.groupby('uuid')['money_raised'].sum().reset_index()

# Renaming the 'money_raised' column to 'total_investment'
total_investments.rename(columns={'money_raised': 'total_investment'}, inplace=True)

# Display the resulting DataFrame
total_investments


,uuid,total_investment
0,0000c657-a8a0-4994-8406-28194b608c2f,6534018.0
1,00031db7-29cf-4639-a9cd-b3b68a96efd9,1381788.0
2,0004ca35-32d5-4d85-94c1-08b13be18d92,2478839.0
3,0006c816-e8f9-4071-ad0d-ce2f2ca1184c,1100000.0
4,000a8818-2378-4234-bf3b-4ab4fd9ded1e,20000.0
...,...,...
29338,fffb40db-c2ba-4768-88af-d971982c18c4,7910000.0
29339,fffbd8fe-7076-4bf4-b491-d16f45da44a3,2100000.0
29340,fffc3924-fafc-48ab-ae66-d6b8b40f71ab,615500.0
29341,fffe3136-fceb-4e85-8239-3c056af97c38,15068751.0


In [101]:
# Merging 'test_task' with 'total_investments' on the 'uuid' column, including all rows from 'test_task' and matching rows from 'total_investments'
test_task = test_task.merge(total_investments, on='uuid', how='left')

# Viewing the Final DataFrame
test_task

,Unnamed: 0,uuid,website,Name,description,valuation_usd,categories,location,categories_list,cities,regions,countries,total_investment
0,0,5639c94c-408b-450a-9694-df238f6d9da9,https://www.perplexity.ai,Perplexity AI,Perplexity is a search engine platform that us...,520000000.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'san-francisco-california', 'uu...","[Artificial Intelligence (AI), Chatbot, Genera...",San Francisco,California,United States,28700000.0
1,5,99cc593e-96ee-48fe-824a-0ebc12a94a63,https://bitscrunch.com,bitsCrunch,AI-enhanced decentralized data networks delive...,58850000.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'munich-bayern', 'uuid': '6469a...","[Analytics, Artificial Intelligence (AI), Bloc...",Munich,Bayern,Germany,2400000.0
2,8,dc6b7b23-1e15-49ed-accc-5d168c32c368,https://lumalabs.ai,Luma AI,Luma AI is a generative AI startup that enable...,250000000.0,"[{'entity_def_id': 'category', 'permalink': '3...","[{'permalink': 'palo-alto-california', 'uuid':...","[3D Technology, Artificial Intelligence (AI), ...",Palo Alto,California,United States,45534227.0
3,30,de7c1608-59de-46f6-8902-3aa91d881470,https://aleph-alpha.com,Aleph Alpha,Aleph Alpha transforms human-machine interacti...,491239560.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'heidelberg-baden-wurttemberg',...","[Artificial Intelligence (AI), Generative AI, ...",Heidelberg,Baden-Wurttemberg,Germany,136352906.0
4,82,71be95da-83de-42c7-9eeb-b3003dc392b8,https://www.jupiter.money,Jupiter,Jupiter is a digital banking app with one simp...,711000000.0,"[{'entity_def_id': 'category', 'permalink': 'b...","[{'permalink': 'mumbai-maharashtra', 'uuid': '...","[Banking, Consulting, Finance, Financial Servi...",Mumbai,Maharashtra,India,86000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,852,933c3357-ef87-43f3-9179-857b66f758c1,https://markopolo.ai/,Markopolo,Markopolo is a complete paid marketing suite t...,3000000.0,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'wilmington-delaware', 'uuid': ...","[Advertising, Advertising Platforms, Generativ...",Wilmington,Delaware,United States,94136.0
427,872,092d0735-eb4e-4847-ae30-79287e222ded,https://www.medulance.com/,Medulance,Medulance is a GPS-based technology platform t...,12075084.0,"[{'entity_def_id': 'category', 'permalink': 'g...","[{'permalink': 'new-delhi-delhi', 'uuid': '7b2...","[GPS, Medical, Mobile Apps]",New Delhi,Delhi,India,241501.0
428,946,4cebcabf-bfbd-43cb-b58e-e6aa45171ee2,https://limacharlie.io,LimaCharlie,Security Infrastructure as a Service (SIaaS).\...,24000000.0,"[{'entity_def_id': 'category', 'permalink': 'c...","[{'permalink': 'walnut-california', 'uuid': 'a...","[Cloud Security, Cyber Security, Information T...",Walnut,California,United States,NaN
429,951,248eb9ad-61b2-4c55-ae5a-e544dc055bdb,https://updf.com,"Superace Software Technologies Co., Ltd.",Superace Software develops cross-platform docu...,19500836.0,"[{'entity_def_id': 'category', 'permalink': 's...","[{'permalink': 'shenzhen-guangdong', 'uuid': '...",[Software],Shenzhen,Guangdong,China,2328458.0


### Saving Final DataFrame to CSV

In [102]:
# Save the resulting DataFrame to a CSV file
test_task.to_csv(r"C:\Users\Administrator\Downloads\Test\final_test_task.csv", index=False)
